In [2]:
import pickle as pk
import matplotlib as mpl
import matplotlib.pyplot as plt
import mpl_toolkits
# from mpl_toolkits import mplot3d
from matplotlib import cm
import matplotlib.patches as patches
import matplotlib.ticker as ticker
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

import numpy as np
import scipy
import math
import pandas as pd
import os
import itertools
import glob

import torch
torch.set_default_dtype(torch.float64)

%matplotlib inline

class ScalarFormatterForceFormatZero(ticker.ScalarFormatter):
    def _set_format(self):  # Override function that finds format to use
        self.format = '%1.0f'  # Give format here

class ScalarFormatterForceFormatOne(ticker.ScalarFormatter):
    def _set_format(self):
        self.format = '%1.1f'

class ScalarFormatterForceFormat(ticker.ScalarFormatter):
    def _set_format(self):
        self.format = '%1.2f'
        
class ScalarFormatterForceFormatThree(ticker.ScalarFormatter):
    def _set_format(self):
        self.format = '%1.3f'

In [3]:
import sys
sys.path.append(os.path.realpath('/Users/wei/Documents/physics/code/tnpy'))
sys.path.append(os.path.realpath('/Users/wei/Documents/physics/code/tnpy/tnpy'))

%load_ext autoreload
%autoreload all

import tnpy as tp
from tnpy import GTensor
from tnpy.mps import FermiMPS as fMPS

In [4]:
# our fMPS tensor convention
# 
#  0 --<--*--<-- 1
#         ｜2
#         v

gt_dual = (0, 1, 0)
gt_shape = ((3, 3), (3, 3), (2, 2))

fmps = fMPS.rand(n=16, dual=gt_dual, shape=gt_shape)
fmps_dagger = fmps.dagger()
print(fmps.inner_product(fmps_dagger, fmps), fmps.inner_product(fmps, fmps_dagger))

fmps_0 = fMPS.rand(n=16, dual=gt_dual, shape=gt_shape)
print(fmps.inner_product(fmps_0.dagger(), fmps))

lc_mps = fmps.left_canonical()
rc_mps = fmps.right_canonical()

print('left inner product:', fmps.inner_product(lc_mps.dagger(), lc_mps))
print('right inner product:', fmps.inner_product(rc_mps.dagger(), rc_mps))

print('Fidelity:', fmps.fidelity(lc_mps, rc_mps))

tensor(8.19915e+20-13975.93738j) tensor(8.19915e+20+13975.93738j)
tensor(2.40315e+20-9.63662e+18j)
left inner product: tensor(6.00000+4.60280e-17j)
right inner product: tensor(6.00000+2.70483e-16j)
Fidelity: tensor(0.00068-5.22973e-21j)


In [5]:
# test GTensor eigen-problem

my_dual = (0, 1, 1, 0)
my_gds = (0, 1), (2, 3)
my_shape = (2, 2), (3, 2), (2, 2), (3, 2)

gt = GTensor.rand(dual=my_dual, shape=my_shape, cflag=True)
print(gt.dual, gt.shape)

dt = gt.push_blocks()
v_whole_shape = (dt.shape[2:])
v_shape = gt.shape[2:]

v_dual = tuple([d ^ 1 for d in gt.dual[2:]])
print(gt.shape, v_shape, gt.dual)
print(v_dual, v_shape, v_whole_shape)

new_gt = GTensor.extract_blocks(dt, gt.dual, gt.shape)

for key, val in new_gt.blocks().items():
    print(key)
    # print(v)
    print((val-gt.blocks()[key]).norm())

def mv(v):
    # covert to torch.tensor
    tv = torch.from_numpy(v.reshape(v_whole_shape)).cdouble()
    # print(tv.shape)
    # build GTensor from dense tensor
    gtv = GTensor.extract_blocks(tv, v_dual, v_shape)
    # test = gtv.push_blocks()
    # print('test', gtv.shape, tv, (test-tv).norm())
    w = tp.gcontract('abcd,cd->ab', gt, gtv)

    return w.push_blocks().numpy().flatten()

# w = mv(v)
# print(w.shape)

dim_op = math.prod(v_whole_shape)
print(dim_op)
op = scipy.sparse.linalg.LinearOperator(shape=(dim_op, dim_op), matvec=mv)
v_init = GTensor.rand(dual=v_dual, shape=v_shape).push_blocks().numpy().flatten()
vals, vecs = scipy.sparse.linalg.eigs(
        op, k=3, which='LM', v0=v_init, maxiter=None, return_eigenvectors=True)
inds = abs(vals).argsort()[::-1]
sorted_vals, sorted_vecs = vals[inds], vecs[:, inds]

print(sorted_vals)
lam = sorted_vals[0]
eig_v = sorted_vecs[:, 0]
w = mv(eig_v)
print(lam*eig_v, w)

eig_vt = torch.from_numpy(eig_v.reshape(v_whole_shape))
eig_gvt = GTensor.extract_blocks(eig_vt, v_dual, v_shape)

right = lam*eig_gvt
left = tp.gcontract('abcd,cd->ab', gt, eig_gvt)

for key, val in right.blocks().items():
    print(key)
    # print(val)
    # print(left.blocks()[key])
    print((val-left.blocks()[key]).norm())

(0, 1, 1, 0) ((2, 2), (3, 2), (2, 2), (3, 2))
((2, 2), (3, 2), (2, 2), (3, 2)) ((2, 2), (3, 2)) (0, 1, 1, 0)
(0, 1) ((2, 2), (3, 2)) torch.Size([4, 5])
(0, 0, 0, 0)
tensor(0.)
(0, 0, 1, 1)
tensor(0.)
(0, 1, 0, 1)
tensor(0.)
(0, 1, 1, 0)
tensor(0.)
(1, 0, 0, 1)
tensor(0.)
(1, 0, 1, 0)
tensor(0.)
(1, 1, 0, 0)
tensor(0.)
(1, 1, 1, 1)
tensor(0.)
20
[ 5.07632106+5.49196831j -0.67601621-1.01456099j -1.04004405+0.45859571j]
[-1.97989266-0.36706353j -2.89319219-0.19163032j -2.45522401-0.02889539j
  0.        +0.j          0.        +0.j         -2.10154421-0.48144266j
 -2.65997717+0.05430537j -2.01097431-0.78955756j  0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j         -1.88143712-0.67224909j -2.18241112-0.49111602j
  0.        +0.j          0.        +0.j          0.        +0.j
 -2.33671878-0.19570903j -2.52957854-0.3905307j ] [-1.97989266-0.36706353j -2.89319219-0.19163032j -2.45522401-0.02889539j
  0.        +0.j          0.        +0.j  

In [6]:
# test one-site solver

m_dual = (0, 1, 0)
m_shape = (4, 4), (4, 4), (2, 2)
a_shape = (3, 3), (3, 3), (2, 2)
a_whole_shape = tuple([sum(x) for x in a_shape])

def fidelty(m, a, le, re):

    x = tp.gcontract('abc,ad,be,dec->', m.conj(), le.conj(reverse=True), re.conj(reverse=True), a)
    y = tp.gcontract('abc,ad,be,dec->', a.conj(), le, re, m)

    return x*y

def solver(m, le, re):

    def mv(v):
        tv = torch.from_numpy(v.reshape(a_whole_shape)).cdouble()
        gtv = GTensor.extract_blocks(tv, m_dual, a_shape)
        temp = tp.gcontract('abc,ad,be,dec->', m.conj(), le.conj(reverse=True), re.conj(reverse=True), gtv)
        w = temp*tp.gcontract('ad,be,dec->abc', le, re, m)

        return w.push_blocks().numpy().flatten()
    
    dim_op = math.prod(a_whole_shape)
    op = scipy.sparse.linalg.LinearOperator(shape=(dim_op, dim_op), matvec=mv)
    v_init = GTensor.rand(dual=m_dual, shape=a_shape).push_blocks().numpy().flatten()
    # print(v_init.shape, dim_op)
    vals, vecs = scipy.sparse.linalg.eigsh(
        op, k=3, which='LA', v0=v_init, maxiter=None, return_eigenvectors=True)
    inds = np.real(vals).argsort()[::-1]
    sorted_vals, sorted_vecs = vals[inds], vecs[:, inds]
    print(sorted_vals)

    lam = sorted_vals[0]
    eig_v = sorted_vecs[:, 0]
    eig_vt = torch.from_numpy(eig_v.reshape(a_whole_shape))
    eig_gvt = GTensor.extract_blocks(eig_vt, m_dual, a_shape)

    return lam, eig_gvt

m = GTensor.rand(dual=m_dual, shape=m_shape, cflag=True)
a = GTensor.rand(dual=m_dual, shape=a_shape, cflag=True)

m_dagger = m.conj()
print(m.dual, m.shape, m_dagger.dual, m_dagger.shape)

le = GTensor.rand(dual=(0, 1), shape=(a_shape[0], m_shape[0]), cflag=True)
re = GTensor.rand(dual=(1, 0), shape=(a_shape[1], m_shape[1]), cflag=True)

(0, 1, 0) ((4, 4), (4, 4), (2, 2)) (1, 0, 1) ((4, 4), (4, 4), (2, 2))


In [7]:
lam, b = solver(m, le, re)
print(fidelty(m, a, le, re), fidelty(m, b, le, re))

[2.82857674e-14 1.07737034e-14 1.06133329e-14]
tensor(61130.77253+6.43979e-12j) tensor(9.46633e-30+2.36658e-30j)


In [5]:
# test MPS compression
num = 8
D = 8
gt_dual = (0, 1, 0)
ref_fmps = fMPS.rand_obc(n=num, dual=gt_dual, max_shape=(D // 2, D // 2))

In [20]:
def test(mps_0, mps_1):
    print(mps_0.inner_product(mps_0.dagger(), mps_1), mps_0.inner_product(mps_1.dagger(), mps_0))
    return mps_0.inner_product(mps_0.dagger(), mps_1)*mps_0.inner_product(mps_1.dagger(), mps_0)

In [30]:
D = 4
fmps = fMPS.rand_obc(n=num, dual=gt_dual, max_shape=(D // 2, D // 2)).left_canonical()

for i, gt in enumerate(fmps.tensors):
    print(i, gt.shape)

# test its inner product
print(fmps.inner_product(fmps.dagger(), fmps))
print(fmps.inner_product(fmps, fmps.dagger()))

print(fmps.inner_product(fmps.dagger(), ref_fmps))
print(fmps.inner_product(ref_fmps.dagger(), fmps))

print('test', test(fmps, ref_fmps).item())
print('test', test(ref_fmps, fmps).item())

TypeError: super(type, obj): obj must be an instance or subtype of type